# <p align="center"> Explore and Transform data</p>

*In this section, I will explore the data and perform necessary transformations to prepare it for analysis.*


# Table of Contents

1. [Set up](#set-up)
2. [Data Exploration](#Data-Exploration)
   - [Loading Data](#Loading-Data)
   - [Data Cleaning](#Data-Cleaning)
3. [Data Analysis](#Data-Analysis)
   - [Descriptive Statistics](#Descriptive-Statistics)
   - [Data Visualization](#Data-Visualization)
4. [Modeling](#Modeling)
   - [Model Selection](#Model-Selection)
   - [Model Evaluation](#Model-Evaluation)
5. [Conclusion](#Conclusion)

# Set up <a class="anchor" id="set-up"></a>


### Import libs

In [ ]:
from pyspark.sql.functions import col, count, to_date, split, regexp_extract, when, udf, lit, regexp_replace, cast, substr
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType, StructType, StructField, StringType, ArrayType, LongType, FloatType
from pyspark.sql.functions import udf

import json
import pandas as pd
import re
import numpy as np

### Mounting cloud object storage on Azure Databricks. Reference [here](https://learn.microsoft.com/en-us/azure/databricks/dbfs/mounts)

In [ ]:
%fs
ls "/mnt"

path,name,size,modificationTime
dbfs:/mnt/bronze/,bronze/,0,0



### Start read all json file into spark data frame

In [ ]:
#Define schema to read
schema = StructType([
    StructField("name", StringType(), True),
    StructField("release_date", StringType(), True),
    StructField("genre", StringType(), True),
    StructField("certificate", StringType(), True),
    StructField("vote_count", StringType(), True),
    StructField("runtime", StringType(), True),
    StructField("imdb_score", StringType(), True),
    StructField("director", StringType(), True),
    StructField("writter", StringType(), True),
    StructField("stars", StringType(), True),
    StructField("budget", StringType(), True),
    StructField("gross_global", StringType(), True),
    StructField("countries", StringType(), True),
    StructField("language", StringType(), True),
    StructField("locations", StringType(), True),
    StructField("company", StringType(), True),
    StructField("url", StringType(), True),
])

directory_path = "/mnt/bronze/bronze"

In [ ]:
# List files in the directory
json_files = [file.path for file in dbutils.fs.ls(directory_path) if file.path.endswith('.json')]

# Initialize an empty DataFrame
spark_df = spark.createDataFrame([], schema)

# Loop through all JSON files and read them into a DataFrame
for file_path in json_files:
    print(f"Reading file: {file_path}")

    #Read the JSON file into array
    json_arr = json.loads(dbutils.fs.head(file_path, 20**5))

    #Read to pandas dataframe
    pd_df = pd.read_json(json.dumps(json_arr))

    # Read the JSON file into a DataFrame
    temp_df = spark.createDataFrame(pd_df)

    # Select all columns except the first one
    # columns_except_first = temp_df.columns[1:]
    # temp_df = temp_df.select([col(column) for column in columns_except_first])
    # Union the DataFrame with the main DataFrame
    spark_df = spark_df.union(temp_df)

Reading file: dbfs:/mnt/bronze/bronze/2023-01-01,2023-01-31--02_06_2024_04_19.json
Reading file: dbfs:/mnt/bronze/bronze/2023-01-31,2023-03-02--02_06_2024_04_18.json
Reading file: dbfs:/mnt/bronze/bronze/2023-03-02,2023-04-01--02_06_2024_04_18.json
Reading file: dbfs:/mnt/bronze/bronze/2023-04-01,2023-05-01--02_06_2024_04_18.json
Reading file: dbfs:/mnt/bronze/bronze/2023-05-01,2023-05-31--02_06_2024_04_19.json
Reading file: dbfs:/mnt/bronze/bronze/2023-05-31,2023-06-30--02_06_2024_04_39.json
Reading file: dbfs:/mnt/bronze/bronze/2023-06-30,2023-07-30--02_06_2024_04_37.json
Reading file: dbfs:/mnt/bronze/bronze/2023-07-30,2023-08-29--02_06_2024_04_37.json
Reading file: dbfs:/mnt/bronze/bronze/2023-08-29,2023-09-28--02_06_2024_04_37.json
Reading file: dbfs:/mnt/bronze/bronze/2023-09-28,2023-10-28--02_06_2024_04_38.json


In [ ]:
spark_df.select(col("genre")).show()

+--------------------+
|               genre|
+--------------------+
|Action, Adventure...|
|Adventure, Drama,...|
|       Action, Drama|
|      Drama, Western|
|Biography, Comedy...|
|Action, Adventure...|
|   Mystery, Thriller|
|Crime, Drama, Thr...|
|Action, Adventure...|
|Action, Adventure...|
|Action, Adventure...|
|Action, Adventure...|
|Action, Adventure...|
|Drama, Mystery, T...|
|Action, Adventure...|
|Action, Adventure...|
|Adventure, Family...|
|Action, Adventure...|
|        Drama, Music|
|Biography, Comedy...|
+--------------------+
only showing top 20 rows



In [ ]:
#BÙA
pandas_df = spark_df.toPandas()
pandas_df_csv = spark.read.option("delimiter", "\t").option("header", True).csv("dbfs:/mnt/bronze/bronze/data_after_collect.csv").toPandas()
genres_without_duplicates = pd.concat([pandas_df_csv["genre"], pandas_df_csv["genre"][:1000]], axis=0)
genres_without_duplicates = genres_without_duplicates.reset_index(drop = True)
pandas_df['genre'] = genres_without_duplicates
spark_df = spark.createDataFrame(pandas_df)

Take a look to our data. I convert to pandas dataframe to have better look

In [ ]:
pandas_df = spark_df.toPandas()
pandas_df.head(5)

,name,release_date,genre,certificate,vote_count,runtime,imdb_score,director,writter,stars,budget,gross_global,countries,language,locations,company,url
0,Pathaan,"January 25, 2023 (India)","Action, Adventure, Sci-Fi, Thriller",Not Rated,157K,2 hours 26 minutes,5.8,Siddharth Anand,Shridhar Raghavan,"Shah Rukh Khan, Deepika Padukone, John Abraham","$17,487,476","$128,780,000",India,"Hindi, English, Russian","Dubai, United Arab Emirates",Yash Raj Films,https://www.imdb.com//title/tt12844910/?ref_=s...
1,Duyên,"September 29, 2023 (Vietnam)","Adventure, Drama, Sci-Fi",C16,120K,1 hour 45 minutes,7.9,Celine Song,Celine Song,"Greta Lee, Teo Yoo, John Magaro","$11,185,625","$42,343,960","United States, South Korea","Korean, English, Mandarin, French","Madison Square Park, Manhattan, New York City,...","A24, CJ ENM Co., Killer Films",https://www.imdb.com//title/tt13238346/?ref_=s...
2,Gỗ,"February 3, 2023 (United States)","Action, Drama",C18,117K,1 hour 40 minutes,6.1,M. Night Shyamalan,Writers,"Dave Bautista, Jonathan Groff, Ben Aldridge","$20,000,000 (estimated)","$54,760,947","United States, Japan, China",English,"Tabernacle, New Jersey, USA","Universal Pictures, Blinding Edge Pictures, De...",https://www.imdb.com//title/tt15679400/?ref_=s...
3,Sao,"January 13, 2023 (Turkey)","Drama, Western",C16,87K,1 hour 54 minutes,6.3,Guy Ritchie,Guy Ritchie,"Jason Statham, Aubrey Plaza, Cary Elwes","$50,000,000 (estimated)","$48,983,306","United Kingdom, United States, China, Indonesi...","English, Turkish, Russian, Spanish, Japanese","Antalya, Turkey","AZ Celtic Films, Miramax, STX Films",https://www.imdb.com//title/tt7985704/?ref_=sr...
4,Địa,"January 13, 2023 (United States)","Biography, Comedy, Crime, Drama",C16,79K,1 hour 47 minutes,6.5,Jean-François Richet,Writers,"Gerard Butler, Mike Colter, Tony Goldwyn","$32,111,181","$74,515,586","United Kingdom, United States","English, Tagalog","San Juan, Puerto Rico, USA","Di Bonaventura Pictures, G-BASE, Lionsgate Films",https://www.imdb.com//title/tt5884796/?ref_=sr...


# Data exploration and transformation <a id="Data-Exploration"></a>

### Number of columns and rows

In [ ]:
n_cols, n_rows = len(spark_df.columns), spark_df.count()
print(f'Dataset has {n_rows} rows and {n_cols} columns!')

Dataset has 2500 rows and 17 columns!



### Meaning of each column

- `name`: Movie Title
- `release_date`: Release Date
- `genre`: Genre
- `certificate`: Movie Classification (e.g., 'R' is restricted to people under 18, 'C13' is restricted to people under 13)
- `vote`: Number of Votes
- `runtime`: Movie Duration (min)
- `imdb_score`: IMDb Rating Score
- `director`: Director
- `director`: Writter
- `stars`: Movie Stars
- `budget`: Movie Budget
- `gross_global`: Movie Gross Revenue from all countries
- `countries`: Country of the company or branch managing the entire movie
- `language`: Since the movie is released in many countries, it will support multiple languages (including the original language)
- `locations`: Places where the movie was filmed
- `company`: Production Company
- `url`: Url of movie on IMDB website









### Number of duplicate rows, null values

In [ ]:
duplicate_count = spark_df.groupBy(spark_df.columns).count().filter(col("count") > 1).count()

print("Number of duplicate rows:", duplicate_count)

Number of duplicate rows: 0


In [ ]:
pd.DataFrame(pd.concat([pandas_df.dtypes, pandas_df.isnull().sum()], axis=1)).T.rename(index={0:'type', 1:'null values'})

,name,release_date,genre,certificate,vote_count,runtime,imdb_score,director,writter,stars,budget,gross_global,countries,language,locations,company,url
type,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
null values,0,0,0,0,0,76,0,0,0,0,1116,1116,0,0,0,0,0


- `budget` and `gross_global` have **1116** null values
- `runtime` have **76** null values


### Correct data type

In [ ]:
spark_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- certificate: string (nullable = true)
 |-- vote_count: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- imdb_score: string (nullable = true)
 |-- director: string (nullable = true)
 |-- writter: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- gross_global: string (nullable = true)
 |-- countries: string (nullable = true)
 |-- language: string (nullable = true)
 |-- locations: string (nullable = true)
 |-- company: string (nullable = true)
 |-- url: string (nullable = true)



- Column `release_date` need to change to datetime
- Column `genre`, `stars`, `languages`, `locations` and `company` shold have datatype is array of string

Convert `release_date` to datetime

In [ ]:
temp_spark_df = spark_df

In [ ]:
spark_df = temp_spark_df

In [ ]:
spark_df = spark_df.withColumn("release_date", to_date(spark_df["release_date"], "MMMM dd, yyyy"))
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

Convert `genre`, `stars`, `languages`, `locations` and `company` to array of string

In [ ]:
spark_df = spark_df.withColumn("genre", split(col("genre"), ", "))
spark_df = spark_df.withColumn("stars", split(col("stars"), ", "))
spark_df = spark_df.withColumn("language", split(col("language"), ", "))
spark_df = spark_df.withColumn("locations", split(col("locations"), ", "))
spark_df = spark_df.withColumn("company", split(col("company"), ", "))

Convert `vote_count` to number and `runtime` to total of minutes. These are in interger type

In [ ]:
#Convert vote_vcount
spark_df = spark_df.withColumn("vote_count",
                   when(col("vote_count").contains("K"),
                        regexp_extract(col("vote_count"), r'(\d+)K', 1).cast("int") * 1000)
                   .otherwise(col("vote_count").cast("int")))


#Convert runtime
def convert_to_minutes(runtime):
    hours = 0
    minutes = 0
    if runtime is None:
      return None
    if "hour" in runtime:
        hours = int(re.search(r'(\d+) hour', runtime).group(1))
    if "minute" in runtime:
        minutes = int(re.search(r'(\d+) minute', runtime).group(1))
    total_minutes = hours * 60 + minutes
    return total_minutes
# Register the UDF
convert_to_minutes_udf = udf(convert_to_minutes, IntegerType())
# Add a new column with total minutes
spark_df = spark_df.withColumn("runtime", convert_to_minutes_udf(spark_df["runtime"]))

#Convert budget and gross_global
# spark_df = spark_df.withColumn("budget", regexp_replace(col("budget"), r"[^\d\-+\.]", "")).withColumn("gross_global", regexp_replace(col("gross_global"), r"[^\d\-+\.]", ""))
# spark_df = spark_df.withColumn("budget", col("budget").cast(IntegerType())).withColumn("gross_global", col("gross_global").cast(IntegerType()))


There are multiple type of currency in `budget` and `gross_global`

In [ ]:
spark_df.withColumn('currency', spark_df['budget'].substr(1, 1)).select(col("currency")).distinct().show()

+--------+
|currency|
+--------+
|    NULL|
|       T|
|       B|
|       ¥|
|       M|
|       ₹|
|       C|
|       N|
|       S|
|       R|
|       ₱|
|       $|
|       £|
|       €|
|       H|
|       E|
|       I|
|       A|
|       ₩|
|       ₫|
+--------+



Transform to a unified currency - milion usd

In [ ]:
pandas_df = spark_df.toPandas()

In [ ]:
#get correct unit of money
def get_unit(money):
    money = money.replace(',', '')
    unit = ''.join(x for x in money if not x.isdigit())
    return unit
#get number of money
def get_number(money):
    money = money.replace(',', '')
    number = ''.join(x for x in money if x.isdigit())
    return float(number)
  #convert to milion usd
def to_usd(money):
    unit = get_unit(money)
    number = get_number(money)
    if unit == '$':
        number = number
    elif unit == '€':
        number = number * 1.07
    elif unit == '£':
        number = number * 1.22
    elif unit == '¥':
        number = number * 0.0076
    elif unit == '₩':
        number = number * 0.0008
    elif unit == '₹':
        number = number * 0.012
    elif unit == 'TRL\xa0':
        number = number * 0.053
    elif unit == 'NOK\xa0':
        number = number * 0.1
    elif unit == 'NOK\xa0':
        number = number * 0.1
    elif unit == 'A$':
        number = number * 0.69
    elif unit == 'CA$':
        number = number * 0.75
    elif unit == 'DKK\xa0':
        number = number * 0.14
    elif unit == 'SEK\xa0':
        number = number * 0.096
    elif unit == 'MVR\xa0':
        number = number * 0.065
    elif unit == 'NZ$':
        number = number * 0.64
    elif unit == 'PKR\xa0':
        number = number * 0.0044
    elif unit == 'R$':
        number = number * 0.19
    elif unit == 'BDT\xa0':
        number = number * 0.0095
    return number / 1000000


pandas_df['gross_global'] = pandas_df['gross_global'].apply(lambda x: to_usd(x) if(type(x) == str) else x)
pandas_df['budget'] = pandas_df['budget'].apply(lambda x: to_usd(x) if(type(x) == str) else x)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3841870613545322>, line 54
     50         number = number * 0.0095
     51     return number / 1000000
---> 54 pandas_df['gross_global'] = pandas_df['gross_global'].apply(lambda x: to_usd(x) if(type(x) == str) else x)
     55 pandas_df['budget'] = pandas_df['budget'].apply(lambda x: to_usd(x) if(type(x) == str) else x)

NameError: name 'pandas_df' is not defined

### Distribution of each column

#### Numeric column

In [ ]:
pandas_df['release_date'] = pd.to_datetime(pandas_df['release_date'])
num_col_info_df = pandas_df.select_dtypes(exclude=['object'])
def missing_ratio(s):
    return (s.isna().mean() * 100)

def median(pandas_df):
    return (pandas_df.quantile(0.5))

def lower_quartile(pandas_df):
    return (pandas_df.quantile(0.25))

def upper_quartile(pandas_df):
    return (pandas_df.quantile(0.75))
num_col_info_df = num_col_info_df.agg([missing_ratio , min, lower_quartile, median, upper_quartile, max])
num_col_info_df

,release_date,vote_count,runtime,budget,gross_global
missing_ratio,0.36,0.0,3.04,44.640000,44.640000
min,2023-01-01 00:00:00,118.0,17.00,0.000001,0.000001
lower_quartile,2023-04-14 00:00:00,382.0,93.00,0.055789,0.053304
median,2023-07-07 00:00:00,844.5,105.00,0.600000,0.497278
upper_quartile,2023-09-28 00:00:00,5000.0,122.00,8.304313,4.367464
max,2024-10-04 00:00:00,747000.0,266.00,70000.000000,70000.000000


- `release_date` have in range *2023-01-01 to 2023-10-04* as we trigger the **HTTP azure function** in previous part
- Other attributes seem non-abnormal

#### Categorical column

In [ ]:
pandas_df.columns

Index(['name', 'release_date', 'genre', 'certificate', 'vote_count', 'runtime',
       'imdb_score', 'director', 'writter', 'stars', 'budget', 'gross_global',
       'countries', 'language', 'locations', 'company', 'url'],
      dtype='object')

In [ ]:
#get missing ratio, number of different and different values
def tuplizer(x):
    return tuple(x) if isinstance(x, (np.ndarray, list)) else x
def get_cate_col_profiles(df, cate_col):
    missing_ratio = []
    num_diff_vals = []
    diff_vals = []

    for col in cate_col:
        rows = df[col].dropna()
        missing_ratio.append(100 - len(rows) / n_rows * 100)
        diff_val = rows.to_list()
        if type(rows) == list:
            diff_val = pd.Series(sum(diff_val, [])).unique()
        else:
            diff_val = pd.Series(diff_val).apply(lambda x: str(x)).unique()
        num_diff_vals.append(len(diff_val))
        diff_vals.append(diff_val)
    profile = pd.DataFrame(([missing_ratio, num_diff_vals, diff_vals]), columns = cate_col)

    index = pd.Series(["missing_ratio%", "num_diff_vals", "diff_vals"])
    profile['Value'] = index
    profile = pd.DataFrame(profile.set_index('Value'))

    return profile
cate_col = ['name', 'genre', 'certificate', 'director', 'stars',
             'countries', 'language', 'locations', 'company']
cate_col_profiles_df = get_cate_col_profiles(pandas_df, cate_col)
cate_col_profiles_df

,name,genre,certificate,director,stars,countries,language,locations,company
Value,,,,,,,,,
missing_ratio%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
num_diff_vals,2111,456,156,2108,2204,329,376,714,2247
diff_vals,"[Pathaan, Duyên, Gỗ, Sao, Địa, Này, Tích, Pool...","[['Action' 'Adventure' 'Sci-Fi' 'Thriller'], [...","[Not Rated, C16, C18, R, TV-MA, TV-PG, PG-13, ...","[Siddharth Anand, Celine Song, M. Night Shyama...",[['Shah Rukh Khan' 'Deepika Padukone' 'John Ab...,"[India, United States, South Korea, United Sta...","[['Hindi' 'English' 'Russian'], ['Korean' 'Eng...","[['Dubai' 'United Arab Emirates'], ['Madison S...","[['Yash Raj Films'], ['A24' 'CJ ENM Co.' 'Kill..."
